In [1]:
import cv2
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow as tf
from tqdm import tqdm
from pathlib import Path
import os

In [2]:
def build_model(pretrained):
    model = Sequential([
        pretrained,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(),
        metrics=['accuracy']
    )
    
    return model

In [3]:
base_path = Path().resolve().parents[1]
image_gen = ImageDataGenerator(rescale=1./255.)

train_flow = image_gen.flow_from_directory(
    os.path.join(base_path, 'data/realvsfake/train/'),
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

Found 100000 images belonging to 2 classes.


In [4]:
image_gen1 = ImageDataGenerator(rescale=1./255.)

valid_flow = image_gen1.flow_from_directory(
    os.path.join(base_path, 'data/realvsfake/valid/'),
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

Found 20000 images belonging to 2 classes.


In [5]:
densenet = DenseNet121(
    weights=None,
    include_top=False,
    input_shape=(224,224,3)
)
model = build_model(densenet)
model.summary()

# from keras.layers import Dense, GlobalAveragePooling2D, Input
# from keras.models import Model

# input_shape = (224, 224, 3)
# inputs = Input(shape=input_shape)
# densenet = DenseNet121(weights=None, include_top=False, input_tensor=inputs)
# x = GlobalAveragePooling2D()(densenet.output)
# outputs = Dense(2, activation='softmax')(x)  # Assuming 2 classes
# model = Model(inputs=inputs, outputs=outputs)
# model = build_model(densenet)
# model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ densenet121 (Functional)        │ ?                      │     7,037,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,037,504 (26.85 MB)

 Trainable params: 6,953,856 (26.53 MB)

 Non-trainable params: 83,648 (326.75 KB)

In [6]:
train_steps = 100000//1000
valid_steps = 20000//1000

history = model.fit(
    train_flow,
    epochs = 10,
    steps_per_epoch =train_steps,
    validation_data =valid_flow,
    validation_steps = valid_steps
)

Epoch 1/10


c:\Users\DELL\anaconda3\envs\wheat_disease_env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 2477s 24s/step - accuracy: 0.5432 - loss: 0.7491 - val_accuracy: 0.4891 - val_loss: 0.8257
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2403s 24s/step - accuracy: 0.5748 - loss: 0.6803 - val_accuracy: 0.4922 - val_loss: 0.8132
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2405s 24s/step - accuracy: 0.6102 - loss: 0.6638 - val_accuracy: 0.5047 - val_loss: 0.7847
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2403s 24s/step - accuracy: 0.6537 - loss: 0.6197 - val_accuracy: 0.5141 - val_loss: 1.3788
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 17432s 176s/step - accuracy: 0.6679 - loss: 0.6193 - val_accuracy: 0.5539 - val_loss: 0.6911
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2262s 23s/step - accuracy: 0.6877 - loss: 0.5894 - val_accuracy: 0.5141 - val_loss: 1.5547
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2390s 24s/step - accuracy: 0.7098 - loss: 0.5773 - val_accuracy: 0.5531 - val_loss: 1.2870
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2550s 26s/step - accuracy: 0.7245 - loss: 0.5489 - v

In [24]:
import os

model_path = os.path.join(base_path, 'data/results/dfd_model.hdf5')

# # Check if the file exists and remove it
# if os.path.exists(model_path):
#     os.remove(model_path)
#     print(f"Existing model file {model_path} removed.")

# Save the model
model.save(model_path)
print(f"Model saved to {model_path}.")

ValueError: Unable to synchronously create dataset (name already exists)

In [18]:
# test_flow = image_gen1.flow_from_directory(
#     os.path.join(base_path , 'data/realvsfake/test/'),
#     target_size=(224, 224),
#     batch_size=1,
#     shuffle = False,
#     class_mode='binary'
# )
# y_pred = model.predict(test_flow)
# y_test = test_flow.classes

In [19]:
# print("ROC AUC Score:", metrics.roc_auc_score(y_test, y_pred))
# print("AP Score:", metrics.average_precision_score(y_test, y_pred))
# print()
# print(metrics.classification_report(y_test, y_pred > 0.5))

In [27]:
model_path = base_path / 'data' / 'results' / 'dfd_model.keras'
from tensorflow.keras.models import load_model
# model = load_model(str(model_path))

# Load the model with explicit input shape and compile
input_shape = (224, 224, 3)  # Example input shape for ResNet50
n_model = tf.keras.models.load_model(model_path, compile=False, custom_objects={'input_shape': input_shape})

ValueError: Layer 'dense' expected 1 input(s). Received 2 instead.